In [6]:
import tkinter as tk
from tkinter import messagebox, Menu
import random
import math
from turtle import pos

from numpy import matrix

# ==========================================
# 1. KLASSEN UND HILFSFUNKTIONEN
# ==========================================

class Spielstein:
    """Repräsentiert ein Haus, Kirchenschiff oder Kirchturm."""
    def __init__(self, spieler_id, typ, ausrichtung='vertikal'):  # Nur 'vertikal' oder 'horizontal'
        self.spieler = spieler_id
        self.typ = typ             # 'Haus', 'Schiff', 'Turm'
        self.ausrichtung = ausrichtung  # 'vertikal' oder 'horizontal'

    def drehen(self):
        """Dreht den Stein nach einem Zug um 90 Grad (wechselt zwischen vertikal/horizontal)."""
        self.ausrichtung = 'horizontal' if self.ausrichtung == 'vertikal' else 'vertikal'

    def get_farbe(self):
        return '#FF9999' if self.spieler == 1 else '#9999FF'

# --- KONSTANTEN ---
ROWS, COLS = 7, 7
CELL_SIZE = 90
WINDOW_SIZE = ROWS * CELL_SIZE
LINE_WIDTH = 3

OFFSET = 30

# ==========================================
# 2. POPUP FÜR RICHTUNGSWAHL
# ==========================================
class RichtungsWaehler(tk.Toplevel):
    def __init__(self, parent, stein_typ):
        super().__init__(parent)
        self.title("Ausrichtung wählen")
        self.geometry("300x180")
        self.ergebnis = None
        x = parent.winfo_x() + 50; y = parent.winfo_y() + 50
        self.geometry(f"+{x}+{y}")

        tk.Label(self, text=f"Wie soll der {stein_typ} stehen?", 
                font=('Arial', 12)).pack(pady=15)
        btn_font = ('Arial', 10, 'bold')
        
        # Nur zwei Buttons für vertikal/horizontal
        tk.Button(self, text="↕ VERTIKAL (nach oben/unten bewegen)", 
                 command=lambda: self.setze('vertikal'), font=btn_font,
                 height=2, width=30).pack(pady=5)
        
        tk.Button(self, text="↔ HORIZONTAL (nach rechts/links bewegen)", 
                 command=lambda: self.setze('horizontal'), font=btn_font,
                 height=2, width=30).pack(pady=5)

        self.transient(parent); self.grab_set(); self.wait_window(self)

    def setze(self, richtung):
        self.ergebnis = richtung
        self.destroy()

# ==========================================
# 3. HAUPTKLASSE SPIELBRETT
# ==========================================

class Spielbrett(tk.Canvas):

    def __init__(self, master):
        super().__init__(master, width=WINDOW_SIZE + 2 * OFFSET, height=WINDOW_SIZE + 2 * OFFSET, bg='#F0F0F0')
        self.pack()
        self.master = master
        self.ki_aktiv = False
        self.ki_spieler_id = 2   #Spieler 2 ist KI
        self.ki_schwierigkeit = "mittel"  # Standard: mittel
        self.spiel_aktiv = True  # Status ob Spiel läuft

        self.spielfeld_matrix = [[None] * COLS for _ in range(ROWS)]
        self.unplatzierte_steine = {
            1: self._initialisiere_steine(1),
            2: self._initialisiere_steine(2)
        }
        self.pfarrer_pos = (4, 4)
        self.aktueller_spieler = 1
        self.ausgewaehlter_stein_pos = None
        self.spieler_farben = {1: 'Rot', 2: 'Blau'}

        self.draw_grid()
        self.draw_labels()
        self.initialisiere_spielfeld()
        self.bind("<Button-1>", self.handle_click)
        self.update_title()
        self.erstelle_menue()
        self.frage_spielmodus()
        
    def erstelle_menue(self):
        """Erstellt das Menü für das Spiel"""
        menubar = Menu(self.master)
        self.master.config(menu=menubar)
        
        # Spiel-Menü
        spiel_menu = Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Spiel", menu=spiel_menu)
        
        spiel_menu.add_command(label="Neues Spiel", command=self.neues_spiel)
        spiel_menu.add_separator()
        spiel_menu.add_command(label="Spielmodus wechseln", command=self.spielmodus_wechseln)
        spiel_menu.add_command(label="Schwierigkeitsgrad ändern", command=self.schwierigkeitsgrad_aendern)
        spiel_menu.add_separator()
        spiel_menu.add_command(label="Beenden", command=self.beenden)
        
        # Hilfe-Menü
        hilfe_menu = Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Hilfe", menu=hilfe_menu)
        
        hilfe_menu.add_command(label="Spielregeln", command=self.zeige_spielregeln)
        hilfe_menu.add_command(label="Über", command=self.zeige_ueber)
    
    def neues_spiel(self):
        """Startet ein komplett neues Spiel"""
        if not self.spiel_aktiv:
            # Wenn Spiel schon vorbei war, einfach neu starten
            self.spiel_neustarten()
        else:
            # Frage nach Bestätigung, wenn Spiel läuft
            antwort = messagebox.askyesno("Neues Spiel", 
                "Möchtest du wirklich ein neues Spiel starten?\nDas aktuelle Spiel geht verloren.")
            if antwort:
                self.spiel_neustarten()
    
    def spiel_neustarten(self):
        """Setzt das Spiel komplett zurück"""
        # Zurücksetzen aller Variablen
        self.spielfeld_matrix = [[None] * COLS for _ in range(ROWS)]
        self.unplatzierte_steine = {
            1: self._initialisiere_steine(1),
            2: self._initialisiere_steine(2)
        }
        self.pfarrer_pos = (4, 4)
        self.aktueller_spieler = 1
        self.ausgewaehlter_stein_pos = None
        self.spiel_aktiv = True
        
        # Canvas leeren und neu zeichnen
        self.delete("all")
        self.draw_grid()
        self.draw_labels()
        self.initialisiere_spielfeld()
        self.update_title()
        
        # KI-Einstellungen beibehalten, aber Spielmodus neu fragen
        self.frage_spielmodus()
    
    def spielmodus_wechseln(self):
        """Wechselt den Spielmodus (Mensch vs KI / Mensch vs Mensch)"""
        if self.spiel_aktiv:
            antwort = messagebox.askyesno("Spielmodus wechseln",
                "Möchtest du den Spielmodus wirklich wechseln?\nDas aktuelle Spiel wird neu gestartet.")
            if not antwort:
                return
        
        self.spiel_neustarten()
    
    def schwierigkeitsgrad_aendern(self):
        """Ändert den Schwierigkeitsgrad der KI"""
        if self.ki_aktiv:
            self.zeige_schwierigkeitsauswahl()
        else:
            messagebox.showinfo("Schwierigkeitsgrad",
                "Der Schwierigkeitsgrad kann nur geändert werden,\nwenn du gegen die KI spielst.")
    
    def beenden(self):
        """Beendet das Spiel"""
        if self.spiel_aktiv:
            antwort = messagebox.askyesno("Beenden", 
                "Möchtest du das Spiel wirklich beenden?")
            if antwort:
                self.master.quit()
        else:
            self.master.quit()
    
    def zeige_spielregeln(self):
        """Zeigt die Spielregeln an"""
        regeln = """
        LASS DIE KIRCHE IM DORF - Spielregeln
        
        ZIEL DES SPIELS:
        Als erster Spieler 9 eigene Steine in einer zusammenhängenden Gruppe zu haben,
        wobei mindestens ein Turm nicht am Rand steht und mindestens ein Kirchenschiff vorhanden ist.
        
        SPIELABLAUF:
        1. PLATZIERUNGSPHASE:
           - Jeder Spieler hat: 1 Turm, 1 Kirchenschiff, 7 Häuser
           - Türme und Kirchenschiffe müssen auf Ecken platziert werden
           - Häuser dürfen NICHT direkt orthogonal neben eigene Steine platziert werden
        
        2. BEWEGUNGSPHASE:
           - Steine gleiten in ihrer Ausrichtung (vertikal/horizontal)
           - Nach jedem Zug dreht sich der Stein um 90 Grad (wechselt Ausrichtung)
        
        3. PFARRERTAUSCH:
           - Der Pfarrer (in der Mitte) kann mit einem blockierten Stein getauscht werden
           - Nur ein blockierter Stein (keine Zugmöglichkeit) kann getauscht werden
        
        STEIN-TYPEN:
        - Haus: Rechteck, bewegt sich vertikal/horizontal je nach Ausrichtung
        - Schiff: Kreis, bewegt sich vertikal/horizontal je nach Ausrichtung
        - Turm: Dreieck, bewegt sich vertikal/horizontal je nach Ausrichtung
        
        AUSRICHTUNG:
        - Vertikal: Stein kann nach oben/unten gleiten
        - Horizontal: Stein kann nach rechts/links gleiten
        
        Viel Spaß beim Spielen!
        """
        messagebox.showinfo("Spielregeln", regeln)
    
    def zeige_ueber(self):
        """Zeigt Informationen über das Spiel an"""
        info = """
        LASS DIE KIRCHE IM DORF
        
        Ein Strategiespiel für 2 Spieler
        
        Version: 1.1
        Entwickelt mit Python und Tkinter
        
        KI-Schwierigkeitsgrade:
        - Einfach: Zufällige Züge
        - Mittel: Strategische Hausplatzierung
        - Schwer: Fortgeschrittene Strategie
        
        Steuerung:
        - Linksklick: Stein auswählen/platzieren
        - Menü: Spieloptionen
        
        © 2023
        """
        messagebox.showinfo("Über das Spiel", info)
        
    def frage_spielmodus(self):
        antwort = messagebox.askquestion("Spielmodus", "Möchtest du gegen die KI spielen?")
        if antwort == 'yes':
            self.ki_aktiv = True
            self.zeige_schwierigkeitsauswahl()
        else:
            self.ki_aktiv = False

    def zeige_schwierigkeitsauswahl(self):
        """Popup für Schwierigkeitsgrad-Auswahl"""
        popup = tk.Toplevel(self.master)
        popup.title("KI Schwierigkeitsgrad")
        popup.geometry("300x150")
        popup.transient(self.master)
        popup.grab_set()
        
        x = self.master.winfo_x() + 100
        y = self.master.winfo_y() + 100
        popup.geometry(f"+{x}+{y}")
        
        tk.Label(popup, text="Wähle den Schwierigkeitsgrad der KI:", 
                font=('Arial', 12)).pack(pady=10)
        
        # Variable für Radiobuttons
        self.ki_schwierigkeit_var = tk.StringVar(value=self.ki_schwierigkeit)
        
        tk.Radiobutton(popup, text="Einfach", variable=self.ki_schwierigkeit_var, 
                      value="einfach", font=('Arial', 10)).pack(anchor='w', padx=20)
        tk.Radiobutton(popup, text="Mittel", variable=self.ki_schwierigkeit_var, 
                      value="mittel", font=('Arial', 10)).pack(anchor='w', padx=20)
        tk.Radiobutton(popup, text="Schwer", variable=self.ki_schwierigkeit_var, 
                      value="schwer", font=('Arial', 10)).pack(anchor='w', padx=20)
        
        tk.Button(popup, text="OK", command=lambda: self.schwierigkeit_uebernehmen(popup), 
                 width=10).pack(pady=10)

    def schwierigkeit_uebernehmen(self, popup):
        self.ki_schwierigkeit = self.ki_schwierigkeit_var.get()
        popup.destroy()
        self.update_title()

    def _initialisiere_steine(self, spieler_id):
        steine = []
        steine.append(Spielstein(spieler_id, 'Turm', 'vertikal'))
        steine.append(Spielstein(spieler_id, 'Schiff', 'vertikal'))
        for _ in range(7): steine.append(Spielstein(spieler_id, 'Haus', 'vertikal'))
        return steine

    def initialisiere_spielfeld(self):
        self.draw_pfarrer(self.pfarrer_pos[0], self.pfarrer_pos[1])

    # ---------------------------------------------------------
    # LOGIK-KERN: BERECHNUNG DER ERLAUBTEN ZÜGE (VEKTOR)
    # ---------------------------------------------------------

    def hat_eigene_orthogonale_nachbarn(self, row, col, spieler_id):
        nachbarn = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
        for r, c in nachbarn:
            if 1 <= r <= ROWS and 1 <= c <= COLS:
                stein = self.spielfeld_matrix[r-1][c-1]
                if stein is not None and stein.spieler == spieler_id:
                    return True
        return False

    def get_valid_moves(self, spieler_id):
        """Berechnet alle erlaubten Züge (BIDIREKTIONALES GLEITEN)."""
        moves = []
        vorrat = self.unplatzierte_steine[spieler_id]

        # --- PHASE 1: PLATZIEREN ---
        if len(vorrat) > 0:
            naechster_stein = vorrat[0]

            # A) Kirchenteile (Ecken, Diagonal-Check)
            if naechster_stein.typ in ['Turm', 'Schiff']:
                alle_ecken = [(1, 1), (1, 7), (7, 1), (7, 7)]
                partner_map = {(1, 1): (7, 7), (7, 7): (1, 1), (1, 7): (7, 1), (7, 1): (1, 7)}

                for (r, c) in alle_ecken:
                    if self.spielfeld_matrix[r-1][c-1] is None:
                        pr, pc = partner_map[(r, c)]
                        partner_stein = self.spielfeld_matrix[pr-1][pc-1]

                        diagonale_ok = True
                        if partner_stein is not None and partner_stein.spieler != spieler_id:
                            diagonale_ok = False

                        if diagonale_ok:
                            moves.append({'type': 'place', 'pos': (r, c), 'stone_type': naechster_stein.typ})

            # B) Häuser (Frei, keine eigenen Nachbarn)
            else:
                for r in range(1, ROWS + 1):
                    for c in range(1, COLS + 1):
                        if self.spielfeld_matrix[r-1][c-1] is None and (r, c) != self.pfarrer_pos:
                            if not self.hat_eigene_orthogonale_nachbarn(r, c, spieler_id):
                                moves.append({'type': 'place', 'pos': (r, c), 'stone_type': 'Haus'})

        # --- PHASE 2: BEWEGEN (Gleiten in beide Richtungen) ---
        else:
            for r in range(1, ROWS + 1):
                for c in range(1, COLS + 1):
                    stein = self.spielfeld_matrix[r-1][c-1]

                    if stein and stein.spieler == spieler_id:

                        # Bestimme die Bewegungsrichtungen basierend auf der Ausrichtung
                        if stein.ausrichtung == 'vertikal':
                            richtungen = [(0, -1), (0, 1)]  # Oben und Unten
                        else:  # 'horizontal'
                            richtungen = [(1, 0), (-1, 0)]  # Rechts und Links

                        # Prüfe BEIDE Richtungen der Achse
                        for dx, dy in richtungen:
                            distanz = 1
                            while True:
                                ziel_r = r + (dy * distanz)
                                ziel_c = c + (dx * distanz)

                                # 1. Ist Ziel noch auf dem Brett?
                                if not (1 <= ziel_r <= ROWS and 1 <= ziel_c <= COLS):
                                    break # Brettende

                                ziel_inhalt = self.spielfeld_matrix[ziel_r-1][ziel_c-1]

                                # 2. Blockiert durch Pfarrer?
                                if (ziel_r, ziel_c) == self.pfarrer_pos:
                                    break

                                # 3. Blockiert durch anderen Stein?
                                if ziel_inhalt is not None:
                                    break

                                # Feld frei -> Zug hinzufügen
                                moves.append({'type': 'move', 'from': (r, c), 'to': (ziel_r, ziel_c)})

                                # Weiter gleiten
                                distanz += 1

        return moves


    
    # --------------------------
    # SIEGBEDINGUNG
    # --------------------------
    def pruefe_sieg(self, spieler_id):
        """
        Prüft, ob der Spieler die Siegbedingung erfüllt:
        - Alle eigenen Steine zusammenhängend
        - Mindestens ein Turm nicht am Rand
        - Mindestens ein Kirchenschiff vorhanden
        """
        from collections import deque

        # Spielfeldgröße
        ROWS, COLS = len(self.spielfeld_matrix), len(self.spielfeld_matrix[0])

        # Alle eigenen Steine sammeln
        steine = [(r, c) for r in range(ROWS) for c in range(COLS)
              if self.spielfeld_matrix[r][c] and self.spielfeld_matrix[r][c].spieler == spieler_id]

        if not steine:
            return False

        # ----------------------
        # 1. Zusammenhängende Gruppe prüfen (BFS)
        # ----------------------
        besuchte = [[False]*COLS for _ in range(ROWS)]
        queue = deque()
        queue.append(steine[0])
        besuchte[steine[0][0]][steine[0][1]] = True
        count = 1

        while queue:
             r, c = queue.popleft()
             for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:  # orthogonal
                nr, nc = r + dr, c + dc
                if 0 <= nr < ROWS and 0 <= nc < COLS and not besuchte[nr][nc]:
                    s = self.spielfeld_matrix[nr][nc]
                    if s and s.spieler == spieler_id:
                        queue.append((nr, nc))
                        besuchte[nr][nc] = True
                        count += 1

                 # Mindestens 9 Steine müssen zusammenhängen
        if count < 9:
            return False

        # ----------------------
        # 2. Turm und Schiff prüfen
        # ----------------------
        turm_ok = False
        schiff_ok = False
        for r, c in steine:
            s = self.spielfeld_matrix[r][c]
            if s.typ == 'Turm':
                 # Turm darf nicht am Rand stehen
                if 0 < r < ROWS-1 and 0 < c < COLS-1:
                    turm_ok = True
            elif s.typ == 'Schiff':
                schiff_ok = True

        if not turm_ok or not schiff_ok:
            return False

        # ----------------------
        # Alles erfüllt -> Sieg
        # ----------------------
        return True

    # --------------------------
    #  Bedingungen für blokierten Zug
    # --------------------------
    def ist_blockierter_stein(self, row, col):
        """Prüft, ob der Stein an dieser Position blockiert ist (kein Zug möglich)."""
        stein = self.spielfeld_matrix[row-1][col-1]
        if not stein:
            return False
        valid_moves = self.get_valid_moves(stein.spieler)
        for m in valid_moves:
            if m['type'] == 'move' and m['from'] == (row, col):
                return False
        return True
    
    def ist_blockierter_stein_matrix(self, matrix, row, col):
        """Prüft, ob der Stein an (row,col) auf einer gegebenen Matrix blockiert ist."""
        stein = matrix[row-1][col-1]
        if stein is None:
            return False
        # Prüfe vertikal/horizontal auf freie Felder
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            rr, cc = row + dr, col + dc
            if 1 <= rr <= ROWS and 1 <= cc <= COLS:
                if matrix[rr-1][cc-1] is None:
                    return False  # Stein kann sich bewegen
        return True  # Keine freien Nachbarn → blockiert

    # --------------------------
    #   Pfarer Austaush
    # --------------------------

    def tausche_pfarrer_mit(self, row, col):
        """
        Tauscht den Pfarrer mit einem blockierten Stein.
        """
        if not self.ist_blockierter_stein_matrix(self.spielfeld_matrix, row, col):
            messagebox.showwarning("Ungültig", "Dieser Stein ist nicht blockiert!")
            return False

        stein = self.spielfeld_matrix[row-1][col-1]
        if stein is None:
            return False

        pf_r, pf_c = self.pfarrer_pos

        # Lösche Visualisierung
        self.delete(f"piece_{row}_{col}")
        self.delete(f"arrow_{row}_{col}")
        self.delete(f"label_{row}_{col}")
        self.delete("pfarrer")
        self.delete("pfarrer_label")

        # Tausch in Matrix
        self.spielfeld_matrix[pf_r-1][pf_c-1] = stein
        self.spielfeld_matrix[row-1][col-1] = None
        stein.drehen()  # Stein dreht sich nach dem Tausch

        # Update Pfarrer Position
        self.pfarrer_pos = (row, col)

        # Visualisierung
        self.draw_piece(pf_r, pf_c, stein)
        self.draw_pfarrer(row, col)

    # ==========================================
    # KI-METHODEN FÜR VERSCHIEDENE SCHWIERIGKEITSGRADE
    # ==========================================

    def ki_macht_zug(self):
        """Hauptmethode für KI-Zug basierend auf Schwierigkeitsgrad"""
        if not self.spiel_aktiv:
            return
            
        valid_moves = self.get_valid_moves(self.ki_spieler_id)
        if not valid_moves: 
            return

        if self.ki_schwierigkeit == "einfach":
            self.ki_zug_einfach(valid_moves)
        elif self.ki_schwierigkeit == "mittel":
            self.ki_zug_mittel(valid_moves)
        else:  # schwer
            self.ki_zug_schwer(valid_moves)
          
    def ki_zug_einfach(self, valid_moves):

        platzierungs_zuege = [z for z in valid_moves if z["type"] == "place"]
        bewegungs_zuege   = [z for z in valid_moves if z["type"] == "move"]

         # ==============================
         # 1️ PLATZIERUNG → IMMER
        # ==============================
        if platzierungs_zuege:
            bester_score = -float("inf")
            bester_zug = None
            beste_ausrichtung = None

            for zug in platzierungs_zuege:
                # Turm / Schiff sofort
                if zug["stone_type"] in ("Turm", "Schiff"):
                    r, c = zug["pos"]
                    stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)
                    stein.ausrichtung = "vertikal"
                    self.platziere_stein(r, c, stein)
                    self.wechsle_spieler()
                    return

                score, ausrichtung = self.bewerte_haus_platzierung(zug)

                if score > bester_score:
                    bester_score = score
                    bester_zug = zug
                    beste_ausrichtung = ausrichtung

            self.fuehre_ki_zug_aus_einfach(bester_zug, beste_ausrichtung)
            return  

        # ==============================
        # 2️ BEWEGUNG → PFARRER ERLAUBT
        # ==============================
        bester_score = -float("inf")
        bester_zug = None

        for zug in bewegungs_zuege:
            score = self.bewerte_bewegung(zug)

            if score > bester_score:
                bester_score = score
                bester_zug = zug

            pfarrer_zug, pfarrer_score = self.bewerte_pfarrer_tausch()

            if pfarrer_zug and pfarrer_score > bester_score + 5:
                bester_zug = pfarrer_zug
        self.fuehre_ki_zug_aus_einfach(bester_zug, None)


    def bewerte_haus_platzierung(self, zug):
        r, c = zug["pos"]
        score = 0

        # Zentrum
        score += max(0, 6 - (abs(r - 4) + abs(c - 4)))

        # Rand vermeiden
        if r in (1, ROWS) or c in (1, COLS):
            score -= 5

        # Nähe zu eigenen Häusern
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == self.ki_spieler_id:
                    if abs(rr - r) + abs(cc - c) == 2:
                        score += 3
        return score, self.bewerte_ausrichtung((r, c))
    

    def bewerte_bewegung(self, zug):
        (fr, fc) = zug["from"]
        (tr, tc) = zug["to"]
        score = 0

        #  Führt der Zug direkt zum Sieg?
        if self.simulierter_sieg(fr, fc, tr, tc):
             return 1000  # absolute Priorität

        # 2.Blockierten Stein befreien
        if self.ist_blockierter_stein_matrix(self.spielfeld_matrix, fr, fc):
            score += 15

        # 3️ Zentrum bevorzugen
        dist_vorher = abs(fr - 4) + abs(fc - 4)
        dist_nachher = abs(tr - 4) + abs(tc - 4)
        score += (dist_vorher - dist_nachher) * 2

        # 4.Nähe zu eigenen Häusern
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == self.ki_spieler_id:
                    vorher = abs(rr - fr) + abs(cc - fc)
                    nachher = abs(rr - tr) + abs(cc - tc)
                    if nachher < vorher:
                        score += 3

            # 5️.Rand vermeiden
        if fr in (1, ROWS) or fc in (1, COLS):
            score += 4
        if tr in (1, ROWS) or tc in (1, COLS):
            score -= 6
        # 6️.Gegner leicht blockieren
        if self.blockiere_gegner((tr, tc), self.ki_spieler_id):
            score += 5

        return score

    def simulierter_sieg(self, fr, fc, tr, tc, spieler_id=None):
        """Simuliere eine Bewegung von (fr,fc) nach (tr,tc) und prüfe, ob dadurch `spieler_id` sofort gewinnt."""
        if spieler_id is None:
            spieler_id = getattr(self, 'ki_spieler_id', None)

        # Grenzen prüfen
        if not (1 <= fr <= ROWS and 1 <= fc <= COLS and 1 <= tr <= ROWS and 1 <= tc <= COLS):
            return False

        # Keine Bewegung möglich
        if self.spielfeld_matrix[fr-1][fc-1] is None:
            return False

         # Aktuelle Werte speichern
        saved_from = self.spielfeld_matrix[fr-1][fc-1]
        saved_to = self.spielfeld_matrix[tr-1][tc-1]

        # Bewegung simulieren
        self.spielfeld_matrix[fr-1][fc-1] = None
        self.spielfeld_matrix[tr-1][tc-1] = saved_from

        # Sieg prüfen
        win = self.pruefe_sieg(spieler_id)

         # Spielfeld zurücksetzen
        self.spielfeld_matrix[fr-1][fc-1] = saved_from
        self.spielfeld_matrix[tr-1][tc-1] = saved_to

        return bool(win)
    
    
    def bewerte_ausrichtung(self, pos):
        """Gibt die bessere Ausrichtung ('vertikal' oder 'horizontal') für eine Platzierung an.
        Nutzt einfache Heuristiken: freie Nachbarfelder und Nähe zur Mitte.
        """
        r, c = pos
        def score_orient(orient):
            score = 0
            if orient == 'vertikal':
                for dr in (-1, 1):
                    rr = r + dr
                    if 1 <= rr <= ROWS:
                        if self.spielfeld_matrix[rr-1][c-1] is None:
                            score += 5
                        else:
                            score -= 2
                    else:
                        score -= 3
            else:  # horizontal
                for dc in (-1, 1):
                    cc = c + dc
                    if 1 <= cc <= COLS:
                        if self.spielfeld_matrix[r-1][cc-1] is None:
                            score += 5
                    else:
                        score -= 2
                else:
                    score -= 3
            center_r, center_c = (ROWS+1)//2, (COLS+1)//2
            score += max(0, 4 - (abs(r-center_r) + abs(c-center_c)))
            return score

        return 'vertikal' if score_orient('vertikal') >= score_orient('horizontal') else 'horizontal'


    ## --------------------------
    ##   PFARRER TAUSCH BEWERTUNG
    ##----------------------------------
    def bewerte_pfarrer_tausch(self):
        """Prüft alle blockierten Steine und bewertet Pfarrer-Tausch-Züge."""
        bester_score = -float("inf")
        bester_zug = None

        pf_r, pf_c = self.pfarrer_pos
       
        for rr in range(1, ROWS+1):
            for cc in range(1, COLS+1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein is None:
                    continue
                if stein.spieler !=self.ki_spieler_id:
                         continue
                if stein is None or not self.ist_blockierter_stein_matrix(self.spielfeld_matrix, rr, cc):
                    continue

                 # 1️ Prüfe, ob Tausch direkt gewinnt
                if self.simulierter_pfarrer_tausch_sieg(rr, cc):
                    return {"type": "pfarrer", "from": (pf_r, pf_c), "to": (rr, cc)}, 1000
                 # 2️ Temporäre Matrix für Bewertung erstellen
                temp_matrix = [row[:] for row in self.spielfeld_matrix]
                temp_matrix[pf_r-1][pf_c-1] = temp_matrix[rr-1][cc-1]  # Pfarrer tauscht blockierten Stein
                temp_matrix[rr-1][cc-1] = None
                    # 2️ Bewertungsheuristik: Nähe zum Zentrum
                center_r, center_c = (ROWS+1)//2, (COLS+1)//2
                score = max(0, 4 - (abs(rr - center_r) + abs(cc - center_c)))

                # 3️ Option: Gegner blockieren
                score += self.blockiert_gegner_sim(temp_matrix, rr, cc, self.ki_spieler_id)
                         
            
                    # 1. Welcher Stein-Typ? (Turm > Schiff > Haus)
                if stein.typ == 'Turm' or stein.typ == 'Schiff':
                     score += 50  # TURM BEFREIEN = SEHR WICHTIG!
                
                else:  # Haus
                    score += 10

                if score <= 0:
                    continue
                if score > bester_score:
                    bester_score = score
                    bester_zug = {"type": "pfarrer", "from": (pf_r, pf_c), "to": (rr, cc)}

        return bester_zug, bester_score
    def blockiert_gegner_sim(self, matrix, tr, tc, spieler_id):
        saved_to = matrix[tr-1][tc-1]
        temp = Spielstein(spieler_id, 'Haus', 'vertikal')
        matrix[tr-1][tc-1] = temp

        blocked_count = 0
        for rr in range(1, ROWS+1):
            for cc in range(1, COLS+1):
                s = matrix[rr-1][cc-1]
                if s is None:
                    continue
                owner = getattr(s, 'spieler', getattr(s, 'spieler_id', getattr(s, 'owner', None)))
                if owner is None or owner == spieler_id:
                    continue
                if self.ist_blockierter_stein_matrix(matrix, rr, cc):
                    blocked_count += 1
        matrix[tr-1][tc-1] = saved_to
        return blocked_count * 20

    def blockiert_gegner(self, tr, tc, spieler_id=None):
        """Simuliere das Setzen/Bewegen eines eigenen Steins nach (tr,tc) und bewerte, 
        wie viele gegnerische Steine dadurch blockiert würden.
        Nutzt die beste Ausrichtung des Steins (vertikal oder horizontal).
            Gibt einen gewichteten Score zurück (höher = besser zum Blockieren)."""

        if spieler_id is None:
            spieler_id = getattr(self, 'ki_spieler_id', None)

        # Grenzen prüfen
        if not (1 <= tr <= ROWS and 1 <= tc <= COLS):
            return 0

        saved_to = self.spielfeld_matrix[tr-1][tc-1]

        try:
            # Beste Ausrichtung bestimmen
            orientierung = self.bewerte_ausrichtung((tr, tc))

            # Temporären Stein mit optimaler Ausrichtung setzen
            temp = Spielstein(spieler_id, 'Haus', orientierung)
            self.spielfeld_matrix[tr-1][tc-1] = temp

            blocked_count = 0
            for rr in range(1, ROWS+1):
                for cc in range(1, COLS+1):
                    s = self.spielfeld_matrix[rr-1][cc-1]
                    if s is None:
                        continue
                    owner = getattr(s, 'spieler', getattr(s, 'spieler_id', getattr(s, 'owner', None)))
                    if owner is None or owner == spieler_id:
                        continue
                    if self.ist_blockierter_stein_matrix(self.spielfeld_matrix, rr, cc):
                        blocked_count += 1

        finally:
            # Spielfeld zurücksetzen
            self.spielfeld_matrix[tr-1][tc-1] = saved_to

            # Gewichtung: jede geblockte Figur zählt +20
        return blocked_count * 20

    ## --------------------------
    ##   SIMULIERTER PFARRER TAUSCH SIEG
    ##----------------------------------
    def simulierter_pfarrer_tausch_sieg(self, row, col):
        pf_r, pf_c = self.pfarrer_pos
        stein = self.spielfeld_matrix[row-1][col-1]
        if not stein:
            return False

            # Tausch simulieren
        self.spielfeld_matrix[pf_r-1][pf_c-1] = stein
        self.spielfeld_matrix[row-1][col-1] = None
        self.pfarrer_pos = (row, col)

        win = self.pruefe_sieg(self.ki_spieler_id)
        # Rücksetzen
        self.spielfeld_matrix[pf_r-1][pf_c-1] = None
        self.spielfeld_matrix[row-1][col-1] = stein
        self.pfarrer_pos = (pf_r, pf_c)
        return win

    def fuehre_ki_zug_aus_einfach(self, zug, ausrichtung=None):

        if zug is None:
            return  # Sicherheitsnetz

        # FALL 1: Platzierung (nur Häuser landen hier!)
        if zug["type"] == "place":
            r, c = zug["pos"]
            stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)

            if ausrichtung:
                stein.ausrichtung = ausrichtung

            self.platziere_stein(r, c, stein)

            # FALL 2: Bewegung
        elif zug["type"] == "move":
            fr, fc = zug["from"]
            tr, tc = zug["to"]
            self.bewege_stein(fr, fc, tr, tc)
        
        elif zug["type"] == "pfarrer":
            # Pfarrer-Tausch: spezieller Ablauf
            to_r, to_c = zug["to"]        
            self.tausche_pfarrer_mit(to_r, to_c)
            # Nach jedem KI-Zug
        if self.pruefe_sieg(self.ki_spieler_id):
            self.spiel_beenden("KI (einfach) hat gewonnen!")
        else:
            self.wechsle_spieler()


    def bewege_stein(self, from_row, from_col, to_row, to_col):
        """Bewegt einen Stein von (from_row, from_col) nach (to_row, to_col) und dreht ihn."""
        stein = self.spielfeld_matrix[from_row-1][from_col-1]
        if stein is None:
            return

        # Lösche Visualisierung
        self.delete(f"piece_{from_row}_{from_col}")
        self.delete(f"arrow_{from_row}_{from_col}")
        self.delete(f"label_{from_row}_{from_col}")

        # Update Matrix
        self.spielfeld_matrix[to_row-1][to_col-1] = stein
        self.spielfeld_matrix[from_row-1][from_col-1] = None

        # Stein drehen
        stein.drehen()

        # Neue Visualisierung
        self.draw_piece(to_row, to_col, stein)
    def ki_zug_mittel(self, valid_moves):
        """Mittlere KI: Verbesserte strategische Logik"""
        bewertete_zuege = []
        
        for zug in valid_moves:
            score = 0
            
            if zug['type'] == 'place':
                r, c = zug['pos']
                
                if zug['stone_type'] in ['Turm', 'Schiff']:
                    # Kirchenteile: Einfache Logik
                    score = 10  # Basiswert
                    
                    # Prüfe diagonales Feld
                    partner_map = {(1,1): (7,7), (7,7): (1,1), (1,7): (7,1), (7,1): (1,7)}
                    pr, pc = partner_map[(r, c)]
                    partner_stein = self.spielfeld_matrix[pr-1][pc-1]
                    
                    if partner_stein and partner_stein.spieler != self.ki_spieler_id:
                        score -= 20  # Vermeiden wenn Gegner diagonal steht
                    
                    # Beste Ausrichtung basierend auf Bewegungsfreiheit
                    beste_richtung = self.finde_beste_ausrichtung_fuer_position((r, c))
                    bewertete_zuege.append((score, zug, beste_richtung))
                else:
                    # Häuser: Strategische Bewertung
                    score += self.strategische_haus_platzierung((r, c), self.ki_spieler_id)
                    
                    # Blockiere Gegner
                    score += self.blockiere_gegner((r, c), self.ki_spieler_id)
                    
                    # Beste Ausrichtung
                    beste_richtung = self.finde_beste_ausrichtung_fuer_position((r, c))
                    bewertete_zuege.append((score, zug, beste_richtung))
            else:
                # Bewegungen bewerten
                r, c = zug['to']
                score += self.strategische_position_bewertung(r, c, self.ki_spieler_id)
                bewertete_zuege.append((score, zug, None))
        
        # Besten Zug wählen
        if bewertete_zuege:
            bewertete_zuege.sort(key=lambda x: x[0], reverse=True)
            
            # Top 3 mit Wahrscheinlichkeitsverteilung
            top_anzahl = min(3, len(bewertete_zuege))
            top_zuege = bewertete_zuege[:top_anzahl]
            
            # Wahrscheinlichkeiten: 60% bester, 30% zweiter, 10% dritter
            rand = random.random()
            if rand < 0.6:
                idx = 0
            elif rand < 0.9:
                idx = 1 if len(top_zuege) > 1 else 0
            else:
                idx = 2 if len(top_zuege) > 2 else 0
            
            bester_score, bester_zug, beste_richtung = top_zuege[idx]
            
            if bester_zug:
                if bester_zug['type'] == 'place':
                    r, c = bester_zug['pos']
                    stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)
                    stein.ausrichtung = beste_richtung if beste_richtung else random.choice(['vertikal', 'horizontal'])
                    self.platziere_stein(r, c, stein)
                else:
                    self.ausgewaehlter_stein_pos = bester_zug['from']
                    self.fuehre_zug_aus(bester_zug['to'][0], bester_zug['to'][1])
                
                self.ausgewaehlter_stein_pos = None
                if self.pruefe_sieg(self.ki_spieler_id):
                    self.spiel_beenden(f"KI ({self.ki_schwierigkeit}) hat gewonnen!")
                else:
                    self.wechsle_spieler()

    def ki_zug_schwer(self, valid_moves):
        """Schwere KI: Erweiterte strategische Bewertung"""
        # Vereinfachte Version - kann später mit MiniMax erweitert werden
        if not valid_moves:
            return
        
        # Verwende zunächst die mittlere KI-Logik mit strengeren Kriterien
        bewertete_zuege = []
        
        for zug in valid_moves:
            score = 0
            
            if zug['type'] == 'place':
                r, c = zug['pos']
                
                if zug['stone_type'] in ['Turm', 'Schiff']:
                    # Kirchenteile mit höheren Gewichtungen
                    score = 15
                    
                    partner_map = {(1,1): (7,7), (7,7): (1,1), (1,7): (7,1), (7,1): (1,7)}
                    pr, pc = partner_map[(r, c)]
                    partner_stein = self.spielfeld_matrix[pr-1][pc-1]
                    
                    if partner_stein and partner_stein.spieler != self.ki_spieler_id:
                        score -= 30  # Stärker vermeiden
                    
                    # Optimale Ausrichtung berechnen
                    beste_richtung = self.optimale_ausrichtung_berechnen((r, c))
                    bewertete_zuege.append((score, zug, beste_richtung))
                else:
                    # Häuser mit erweiterter Bewertung
                    score += self.erweiterte_haus_bewertung((r, c), self.ki_spieler_id)
                    score += self.blockiere_gegner((r, c), self.ki_spieler_id) * 1.5
                    
                    # Zentrum stärker gewichten
                    dist_zentrum = abs(r - 4) + abs(c - 4)
                    score += max(0, 15 - dist_zentrum * 2)
                    
                    # Optimale Ausrichtung
                    beste_richtung = self.optimale_ausrichtung_berechnen((r, c))
                    bewertete_zuege.append((score, zug, beste_richtung))
            else:
                # Bewegungen mit erweiterter Bewertung
                r, c = zug['to']
                score += self.erweiterte_bewegungs_bewertung(zug, self.ki_spieler_id)
                bewertete_zuege.append((score, zug, None))
        
        # Immer besten Zug wählen (keine Zufälligkeit)
        if bewertete_zuege:
            bewertete_zuege.sort(key=lambda x: x[0], reverse=True)
            bester_score, bester_zug, beste_richtung = bewertete_zuege[0]
            
            if bester_zug:
                if bester_zug['type'] == 'place':
                    r, c = bester_zug['pos']
                    stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)
                    stein.ausrichtung = beste_richtung if beste_richtung else 'vertikal'
                    self.platziere_stein(r, c, stein)
                else:
                    self.ausgewaehlter_stein_pos = bester_zug['from']
                    self.fuehre_zug_aus(bester_zug['to'][0], bester_zug['to'][1])
                
                self.ausgewaehlter_stein_pos = None
                if self.pruefe_sieg(self.ki_spieler_id):
                    self.spiel_beenden(f"KI ({self.ki_schwierigkeit}) hat gewonnen!")
                else:
                    self.wechsle_spieler()

    def finde_beste_ausrichtung_fuer_position(self, pos):
        """Findet die beste Ausrichtung für eine Position"""
        r, c = pos
        
        # Zähle freie Felder in vertikaler Richtung
        freie_vertikal = 0
        for dist in range(1, 7):
            if 1 <= r - dist <= ROWS:
                if self.spielfeld_matrix[r-dist-1][c-1] is None and (r-dist, c) != self.pfarrer_pos:
                    freie_vertikal += 1
                else:
                    break
        
        for dist in range(1, 7):
            if 1 <= r + dist <= ROWS:
                if self.spielfeld_matrix[r+dist-1][c-1] is None and (r+dist, c) != self.pfarrer_pos:
                    freie_vertikal += 1
                else:
                    break
        
        # Zähle freie Felder in horizontaler Richtung
        freie_horizontal = 0
        for dist in range(1, 7):
            if 1 <= c - dist <= COLS:
                if self.spielfeld_matrix[r-1][c-dist-1] is None and (r, c-dist) != self.pfarrer_pos:
                    freie_horizontal += 1
                else:
                    break
        
        for dist in range(1, 7):
            if 1 <= c + dist <= COLS:
                if self.spielfeld_matrix[r-1][c+dist-1] is None and (r, c+dist) != self.pfarrer_pos:
                    freie_horizontal += 1
                else:
                    break
        
        # Wähle die Richtung mit mehr freien Feldern
        if freie_vertikal >= freie_horizontal:
            return 'vertikal'
        else:
            return 'horizontal'

    def optimale_ausrichtung_berechnen(self, pos):
        """Erweiterte Berechnung der optimalen Ausrichtung"""
        # Vereinfachte Version - ähnlich wie finde_beste_ausrichtung_fuer_position
        return self.finde_beste_ausrichtung_fuer_position(pos)

    def spiel_beenden(self, nachricht):
        """Beendet das Spiel mit einer Nachricht"""
        self.spiel_aktiv = False
        antwort = messagebox.askyesno("Spiel vorbei", f"{nachricht}\n\nMöchtest du ein neues Spiel starten?")
        if antwort:
            self.spiel_neustarten()
        else:
            # Titel aktualisieren, um anzuzeigen dass Spiel vorbei ist
            self.master.title(f"Lass die Kirche im Dorf - Spiel beendet")

    # ==========================================
    # BEWERTUNGSFUNKTIONEN
    # ==========================================

    def strategische_haus_platzierung(self, pos, spieler_id):
        """Strategische Bewertung für Haus-Platzierung"""
        r, c = pos
        score = 0
        
        # 1. Distanz zu eigenen Steinen
        min_abstand = 999
        eigene_steine_pos = []
        
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == spieler_id:
                    eigene_steine_pos.append((rr, cc))
                    dist = abs(rr - r) + abs(cc - c)
                    if dist < min_abstand and dist > 0:
                        min_abstand = dist
        
        if 2 <= min_abstand <= 3:
            score += 15
        elif min_abstand == 4:
            score += 5
        elif min_abstand >= 5:
            score -= 10
        
        # 2. Diagonale Nachbarn für Cluster-Bildung
        diagonale_nachbarn = [(r-1, c-1), (r-1, c+1), (r+1, c-1), (r+1, c+1)]
        diagonale_eigene = 0
        for dr, dc in diagonale_nachbarn:
            if 1 <= dr <= ROWS and 1 <= dc <= COLS:
                stein = self.spielfeld_matrix[dr-1][dc-1]
                if stein and stein.spieler == spieler_id:
                    diagonale_eigene += 1
        
        if diagonale_eigene >= 2:
            score += 20
        elif diagonale_eigene == 1:
            score += 8
        else:
            score -= 5
        
        # 3. Zentrum bevorzugen
        dist_zentrum = abs(r - 4) + abs(c - 4)
        score += max(0, 8 - dist_zentrum)
        
        # 4. Rand meiden
        if r == 1 or r == ROWS or c == 1 or c == COLS:
            score -= 12
        
        return score

    def blockiere_gegner(self, pos, spieler_id):
        """Bewertet Blockadewirkung auf Gegner"""
        r, c = pos
        score = 0
        gegner_id = 1 if spieler_id == 2 else 2
        
        # Schau nach Häusern des Gegners
        gegner_haeuser = []
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == gegner_id and stein.typ == 'Haus':
                    gegner_haeuser.append((rr, cc))
        
        # Blockiere Gegnerhäuser
        for hr, hc in gegner_haeuser:
            dist = abs(hr - r) + abs(hc - c)
            if 2 <= dist <= 3:
                score += 15
            elif dist == 1:
                score += 25
        
        return score

    def strategische_position_bewertung(self, r, c, spieler_id):
        """Bewertet strategische Position für Bewegungen"""
        score = 0
        
        # Verbindung zu eigenen Steinen
        eigene_nachbarn = 0
        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = r + dr, c + dc
            if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                stein = self.spielfeld_matrix[nr-1][nc-1]
                if stein and stein.spieler == spieler_id:
                    eigene_nachbarn += 1
        
        if eigene_nachbarn >= 2:
            score += 25
        elif eigene_nachbarn == 1:
            score += 10
        
        # Distanz zum Zentrum
        dist_zentrum = abs(r - 4) + abs(c - 4)
        score += max(0, 10 - dist_zentrum * 2)
        
        return score

    def erweiterte_haus_bewertung(self, pos, spieler_id):
        """Erweiterte Bewertung für Häuser (schwere KI)"""
        score = self.strategische_haus_platzierung(pos, spieler_id) * 1.5
        
        # Zusätzliche Gewichtung für Cluster-Bildung
        r, c = pos
        cluster_potential = 0
        
        # Prüfe 3x3 Bereich um die Position
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                if dr == 0 and dc == 0:
                    continue
                nr, nc = r + dr, c + dc
                if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                    stein = self.spielfeld_matrix[nr-1][nc-1]
                    if stein and stein.spieler == spieler_id:
                        cluster_potential += 10
        
        score += cluster_potential
        
        return score

    def erweiterte_bewegungs_bewertung(self, zug, spieler_id):
        """Erweiterte Bewertung für Bewegungen (schwere KI)"""
        from_r, from_c = zug['from']
        to_r, to_c = zug['to']
        score = 0
        
        # Verbesserung der Position?
        score_to = self.strategische_position_bewertung(to_r, to_c, spieler_id)
        score_from = self.strategische_position_bewertung(from_r, from_c, spieler_id)
        score += (score_to - score_from) * 2
        
        # Blockiere ich mich selbst?
        if self.ist_blockierter_stein_matrix(self.spielfeld_matrix, from_r, from_c):
            score += 30
        
        # Entfernung vom eigenen Stein verringern?
        min_dist_vorher = self.minimale_distanz_zu_eigenen(from_r, from_c, spieler_id)
        min_dist_nachher = self.minimale_distanz_zu_eigenen(to_r, to_c, spieler_id)
        
        if min_dist_nachher < min_dist_vorher:
            score += 20
        
        return score

    def minimale_distanz_zu_eigenen(self, r, c, spieler_id):
        """Berechnet minimale Distanz zu eigenen Steinen"""
        min_dist = 999
        
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == spieler_id:
                    dist = abs(rr - r) + abs(cc - c)
                    if dist < min_dist:
                        min_dist = dist
        
        return min_dist if min_dist < 999 else 0

    # ==========================================
    # INTERAKTION
    # ==========================================

    def handle_click(self, event):
        if not self.spiel_aktiv:
            return
            
        row, col = self.get_grid_coords(event.x, event.y)
        if row is None: return

        valid_moves = self.get_valid_moves(self.aktueller_spieler)
        is_placement_phase = any(m['type'] == 'place' for m in valid_moves)
        stein_in_zelle = self.spielfeld_matrix[row-1][col-1]

        # =========================================================
        # 1️.PFARRER WIRD ANGEKLICKT → PFARRER-MODUS START
        # =========================================================
        if (row, col) == self.pfarrer_pos and not is_placement_phase:
            self.ausgewaehlter_stein_pos = 'pfarrer'
            messagebox.showinfo(
                "Pfarrer",
                "Wähle jetzt einen blockierten Stein zum Tauschen."
            )
            return

        # --- FALL A: PLATZIEREN ---
        if is_placement_phase:
            move_match = None
            for move in valid_moves:
                if move['type'] == 'place' and move['pos'] == (row, col):
                    move_match = move
                    break

            if move_match:
                stein_typ = self.unplatzierte_steine[self.aktueller_spieler][0].typ
                waehler = RichtungsWaehler(self.master, stein_typ)
                gewaehlte_richtung = waehler.ergebnis
                if gewaehlte_richtung:
                    neues_teil = self.unplatzierte_steine[self.aktueller_spieler].pop(0)
                    neues_teil.ausrichtung = gewaehlte_richtung
                    self.platziere_stein(row, col, neues_teil)
                    if self.pruefe_sieg(self.aktueller_spieler):
                        self.spiel_beenden(f"Spieler {self.aktueller_spieler} hat gewonnen!")
                        return
                    self.wechsle_spieler()
                else: messagebox.showinfo("Info", "Abgebrochen.")
            else:
                self.zeige_platzierungs_fehler(row, col, valid_moves)

        # --- FALL B: BEWEGEN ---
        else:
            # 1.PFARRER-MODUS
            if self.ausgewaehlter_stein_pos == 'pfarrer':
                if self.ist_blockierter_stein_matrix(self.spielfeld_matrix, row, col):
                    self.tausche_pfarrer_mit(row, col)
                    self.wechsle_spieler()
                else:
                     messagebox.showwarning(
                        "Ungültig",
                        "Du musst einen blockierten Stein auswählen."
                    )

                         # zurücksetzen!
                self.ausgewaehlter_stein_pos = None
                return
            if not self.ausgewaehlter_stein_pos:
                if stein_in_zelle and stein_in_zelle.spieler == self.aktueller_spieler:
                    kann_ziehen = any(m['type'] == 'move' and m['from'] == (row, col) for m in valid_moves)
                    if kann_ziehen:
                        self.ausgewaehlter_stein_pos = (row, col)
                        self.markiere_stein(row, col, True)
                    else: messagebox.showwarning("Blockiert", "Dieser Stein kann nicht ziehen (Weg versperrt oder Brettende).")
            else:
                start_r, start_c = self.ausgewaehlter_stein_pos
                if (row, col) == (start_r, start_c):
                    self.markiere_stein(start_r, start_c, False)
                    self.ausgewaehlter_stein_pos = None
                    return

                zug_valide = any(m['type'] == 'move' and m['from'] == (start_r, start_c) and m['to'] == (row, col) for m in valid_moves)
                if zug_valide:
                    self.fuehre_zug_aus(row, col)
                    if self.pruefe_sieg(self.aktueller_spieler):
                        self.spiel_beenden(f"Spieler {self.aktueller_spieler} hat gewonnen!")
                        return
                    self.ausgewaehlter_stein_pos = None
                    self.wechsle_spieler()
                else: messagebox.showwarning("Ungültig", "Hierhin kann der Stein nicht ziehen.")

    def zeige_platzierungs_fehler(self, r, c, valid_moves):
        if len(self.unplatzierte_steine[self.aktueller_spieler]) > 0:
            typ = self.unplatzierte_steine[self.aktueller_spieler][0].typ
            if typ in ['Turm', 'Schiff']:
                messagebox.showwarning("Regel", "Kirchenteile müssen auf eine freie Ecke!")
            elif self.spielfeld_matrix[r-1][c-1] is not None: pass
            elif self.hat_eigene_orthogonale_nachbarn(r, c, self.aktueller_spieler):
                messagebox.showwarning("Regel", "Nicht direkt neben eigene Steine bauen!")
            else: messagebox.showinfo("Info", "Ungültiges Feld.")

    def fuehre_zug_aus(self, ziel_row, ziel_col):
        start_row, start_col = self.ausgewaehlter_stein_pos
        stein = self.spielfeld_matrix[start_row-1][start_col-1]

        self.delete(f"piece_{start_row}_{start_col}")
        self.delete(f"arrow_{start_row}_{start_col}")
        self.delete(f"label_{start_row}_{start_col}")

        self.spielfeld_matrix[start_row-1][start_col-1] = None
        self.spielfeld_matrix[ziel_row-1][ziel_col-1] = stein

        stein.drehen()
        self.draw_piece(ziel_row, ziel_col, stein)

    def platziere_stein(self, row, col, stein_obj):
        self.spielfeld_matrix[row-1][col-1] = stein_obj
        self.draw_piece(row, col, stein_obj)

    def wechsle_spieler(self):
        self.aktueller_spieler = 1 if self.aktueller_spieler == 2 else 2
        self.update_title()
        # KI ist dran?
        if self.ki_aktiv and self.aktueller_spieler == self.ki_spieler_id and self.spiel_aktiv:
            self.master.after(500, self.ki_macht_zug)

    def update_title(self):
        vorrat = self.unplatzierte_steine[self.aktueller_spieler]
        if len(vorrat) > 0: phase = f"Platzieren: {vorrat[0].typ}"
        else: phase = "Bewegen"
        ki_info = f" | KI: {self.ki_schwierigkeit}" if self.ki_aktiv else ""
        status = " (Spiel aktiv)" if self.spiel_aktiv else " (Spiel beendet)"
        self.master.title(f"Lass die Kirche im Dorf | Am Zug: {self.spieler_farben[self.aktueller_spieler]} | Phase: {phase}{ki_info}{status}")

    # --- VISUALISIERUNG ---
    def draw_grid(self):
        for i in range(ROWS + 1):
            p = i * CELL_SIZE + OFFSET
            self.create_line(OFFSET, p, WINDOW_SIZE + OFFSET, p, width=LINE_WIDTH, fill="#404040")
            self.create_line(p, OFFSET, p, WINDOW_SIZE + OFFSET, width=LINE_WIDTH, fill="#404040")

    def draw_labels(self):
        for i in range(7):
            c = (i + 0.5) * CELL_SIZE + OFFSET
            self.create_text(c, OFFSET/2, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))
            self.create_text(OFFSET/2, c, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))

    def get_canvas_coords(self, row, col):
        return (col - 0.5) * CELL_SIZE + OFFSET, (row - 0.5) * CELL_SIZE + OFFSET

    def get_grid_coords(self, x, y):
        if x < OFFSET or y < OFFSET: return None, None
        c, r = int((x-OFFSET)//CELL_SIZE)+1, int((y-OFFSET)//CELL_SIZE)+1
        if 1<=r<=ROWS and 1<=c<=COLS: return r, c
        return None, None

    def draw_pfarrer(self, r, c):
        x, y = self.get_canvas_coords(r, c)
        rad = CELL_SIZE * 0.2
        self.create_oval(x-rad, y-rad, x+rad, y+rad, fill='#333', outline='black', width=2, tags="pfarrer")
        self.create_text(x, y, text="P", fill='white', font=('Arial', 14, 'bold'), tags="pfarrer_label")

    def draw_piece(self, r, c, stein):
        x_c, y_c = self.get_canvas_coords(r, c)
        farbe = stein.get_farbe()
        rad = CELL_SIZE * 0.38

        if stein.typ == 'Haus':
            self.create_rectangle(x_c - rad, y_c - rad, x_c + rad, y_c + rad, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")
        elif stein.typ == 'Schiff':
            self.create_oval(x_c - rad, y_c - rad, x_c + rad, y_c + rad, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")
        elif stein.typ == 'Turm':
            points = [x_c, y_c - rad, x_c + rad, y_c + rad, x_c - rad, y_c + rad]
            self.create_polygon(points, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")

        if stein.typ != 'Pfarrer':
            # VISUALISIERUNG: Doppelpfeil zeigt Bewegungsrichtungen
            arrow_len = CELL_SIZE * 0.35
            
            if stein.ausrichtung == 'vertikal':
                # Doppelpfeil für oben/unten
                self.create_line(x_c, y_c - arrow_len, x_c, y_c + arrow_len,
                                arrow=tk.BOTH, width=4, arrowshape=(10, 12, 5), 
                                fill='black', tags=f"arrow_{r}_{c}")
            else:  # horizontal
                # Doppelpfeil für rechts/links
                self.create_line(x_c - arrow_len, y_c, x_c + arrow_len, y_c,
                                arrow=tk.BOTH, width=4, arrowshape=(10, 12, 5), 
                                fill='black', tags=f"arrow_{r}_{c}")

                
    def markiere_stein(self, r, c, active):
        if self.spielfeld_matrix[r-1][c-1]:
            outline = '#FFFF00' if active else 'black'
            width = 5 if active else 2
            self.itemconfig(f"piece_{r}_{c}", outline=outline, width=width)

if __name__ == '__main__':
    root = tk.Tk()
    root.title("Lass die Kirche im Dorf")
    root.geometry(f"{WINDOW_SIZE + 2*OFFSET + 20}x{WINDOW_SIZE + 2*OFFSET + 20}")
    sb = Spielbrett(root)
    root.mainloop()